In [1]:
import pandas as pd
import ipywidgets as widgets
import csv

In [2]:
ElementDF = pd.read_csv('RuleElements.csv')
SelectorList = pd.read_csv('selectors.csv').columns.to_list()
OptionList = pd.read_csv('options.csv').columns.to_list()
CommonPredicates = ['agile','finesse','rage','unarmed','fear','mental','poison','disease','fear','sweep']
DamageTypes = ['','slashing','piercing','bludgeoning','precision','fire','cold','electricity','acid','poison','sonic','force','mental','lawful','chaotic','good','evil','positive','negative']

In [3]:
def MakeRuleElement(Key,Options):
    ####################################################################
    # This takes in a key and options for that key and builds a rule element.
    #
    # key -- A string giving the key for the rule element.
    # options -- An array of strings containing the options for the rule element
    ####################################################################
    
    RuleElement = '{"key\":\"' + Key + '"'
    
    for Option in Options:
        RuleElement += ',' + Option
    
    RuleElement += '}'
    
    return print(RuleElement)

def GetPropertyList(row):
    ####################################################################
    # This returns a list of the applicable rule elements.
    #
    # To be used with df.apply(), this is a utility function and just
    # helps to store values.  It does not need to be run except to add 
    # new properties.
    ####################################################################
    
    row = row[PropertyList]
    
    passed = row[row == 'y'].index.tolist()
    return(passed)

def PreparePredicates(All, Any, Not):
    ####################################################################
    # This puts the predacate lists together into a single entity
    #
    # All -- List containing predicates which must all be satisfied
    # Any -- List containing predicates where one must be satified.
    # Not -- !All
    ####################################################################
    
    Predicate = str()
    
    if len(All + Any + Not) > 0:
        Predicate += '\"predicate\":{'
    
    if len(All) > 0:
        Predicate += '\"all\":[\"'
        i = 1
        for each in All:
            if i == len(All):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'
        
        if len(Any + Not) > 0:
            Predicate += ','
            
    if len(Any) > 0:
        Predicate += '\"any\":[\"'
        i = 1
        for each in Any:
            if i == len(Any):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'
        
        if len(Not) > 0:
            Predicate += ','
            
    if len(Not) > 0:
        Predicate += '\"not\":[\"'
        i = 1
        for each in Not:
            if i == len(Not):
                Predicate += each + '\"'
            else:
                Predicate += each + '\",\"'
                i += 1
        Predicate += ']'

    if len(All + Any + Not) > 0:
        Predicate += '}'
        
    return Predicate


def PrepareOptions(Selector, AdvancedSelector, Value, Type, DamageType, DieSize, DiceNumber, Predicate, Label, Name, Override):
    ####################################################################
    # This brings each option together and puts them into a single string.
    #
    # All inputs are strings with the exception of value, which is an integer.
    ####################################################################
    
    Options = []
    
    if len(Selector) > 0:
        Options += ['"selector":"' + AdvancedSelector + Selector + '"']
    
    if len(DamageType) > 0:
        if DamageType == 'precision':
            Options += ['"damageCategory":"' + DamageType + '"']
        else:
            Options += ['"damageType":"' + DamageType + '"']

    if len(DieSize) > 0:
        if Override:
            Options += ['"override":{"dieSize":"' + DieSize + '"}']
        else:
            Options += ['"dieSize":"' + DieSize + '"']
        
    if DiceNumber > 0:
        Options += ['"diceNumber":' + str(DiceNumber)]
    
    if abs(Value) > 0:
        Options += ['"value":' + str(Value)]
        
    if len(Type) > 0:
        Options += ['"type":"' + Type + '"']
        
    if len(Predicate) >0:
        Options += [Predicate]
    
    if Label:
        Options += ['"label":"YOUR TEXT HERE"']
    
    if Name:
        Options += ['"name":"YOUR TEXT HERE"']
        
    return Options

In [4]:
PropertyList = SelectorList
ElementDF['ValidSelectors'] = ElementDF.apply(GetPropertyList,axis = 1)

PropertyList = OptionList
ElementDF['ValidOptions'] = ElementDF.apply(GetPropertyList,axis = 1)

In [5]:
Key = widgets.Dropdown(
    options=ElementDF.FriendlyName.tolist(),
    value=ElementDF.FriendlyName.tolist()[0],
    description='Key:',
    disabled=False
)

All = widgets.SelectMultiple(
        options=CommonPredicates,
        value=[],
        description='Predicate:All',
        disabled=False )
Any = widgets.SelectMultiple(
        options=CommonPredicates,
        value=[],
        description='Any',
        disabled=False )
Not = widgets.SelectMultiple(
        options=CommonPredicates,
        value=[],
        description='Not',
        disabled=False )

Selector = widgets.Dropdown(
options=ElementDF.ValidSelectors[ElementDF.FriendlyName == Key.value].values[0],
value='all',
description='Selector',
disabled=False)

AdvancedSelector = widgets.Dropdown(
options=[('no advanced selector',''),('Only this item','{item|_id}-'),('swords','sword-weapon-group-'),('bows','bow-weapon-group-'),('knives','knife-weapon-group-'),('spears','spear-weapon-group-'),('axes','axe-weapon-group-'),('hammers','hammer-weapon-group-'),('picks','pick-weapon-group-'),('flails','flail-weapon-group-'),('brawling','brawling-weapon-group-'),('polearms','polearm-weapon-group-'),('sling','sling-weapon-group-'),('bombs','bomb-weapon-group-'),('shields','shield-weapon-group-'),('darts','dart-weapon-group-'),('clubs','clubs-weapon-group-')],
value='',
description='Advanced Selector',
disabled=False )

Label = widgets.Checkbox(
    value=False,
    description='Add a label?',
    disabled=False,
    indent=False
)

Name = widgets.Checkbox(
    value=False,
    description='Add a Name?',
    disabled=False,
    indent=False
)

DamageType = widgets.Dropdown(
options=DamageTypes,
value='',
description='Damage type',
disabled=False )

Value = widgets.IntText(
value=0,
description='Value:',
disabled=False)

DiceNumber = widgets.IntText(
value=0,
description='Number of dice:',
disabled=False )
DieSize = widgets.Dropdown(
options=['','d4','d6','d8','d10','d12'],
value='',
description='Die size:',
disabled=False )

Override = widgets.Checkbox(
value=False,
description='Override base damage?',
disabled=False,
indent=False )

Type = widgets.Dropdown(
options=['','circumstance','item','status'],
value='',
description='Type',
disabled=False )

In [6]:
display(Key)

KeyButton = widgets.Button(description = "Select Key")
SelectorButton = widgets.Button(description = "Set Selectors")
FinishButton = widgets.Button(description = "Create Rule Element")

def SetKey(b):
    
    Selector.options = ElementDF.ValidSelectors[ElementDF.FriendlyName == Key.value].values[0]
    Selector.value = ElementDF.ValidSelectors[ElementDF.FriendlyName == Key.value].values[0][0]
    
    return display(Selector), display(AdvancedSelector), display(SelectorButton)

def SetSelector(b):
    
    if (('damage' in Selector.value) and (Key.value != 'Striking rune')):
        display(DamageType)

    if 'value' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        display(Value)

    if 'diceNumber' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        display(DiceNumber)

    if 'dieSize' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        display(DieSize)
        display(Override)

    if 'type' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        display(Type)

    if 'predicate' in ElementDF.ValidOptions[ElementDF.FriendlyName == Key.value].values[0]:
        display(All)
        display(Any)
        display(Not)

    display(Label)
    display(Name)
    
    display(FinishButton)
    
    return

def FinishElement(b):

    Predicate = PreparePredicates(All.value, Any.value, Not.value)
    Options = PrepareOptions(Selector.value, AdvancedSelector.value, Value.value, Type.value, DamageType.value, DieSize.value, DiceNumber.value, Predicate, Label.value, Name.value, Override.value)

    MakeRuleElement(ElementDF.Key[ElementDF.FriendlyName == Key.value].values[0],Options)
    
    return

display(KeyButton)

KeyButton.on_click(SetKey)
SelectorButton.on_click(SetSelector)
FinishButton.on_click(FinishElement)

Dropdown(description='Key:', options=('Flat Modifier', 'Damage Dice', 'Weapon Potency', 'Striking rune', 'Mult…

Button(description='Select Key', style=ButtonStyle())

Dropdown(description='Selector', options=('all', 'str-based', 'dex-based', 'con-based', 'int-based', 'wis-base…

Dropdown(description='Advanced Selector', options=(('no advanced selector', ''), ('Only this item', '{item|_id…

Button(description='Set Selectors', style=ButtonStyle())

...

SelectMultiple(description='Any', options=('agile', 'finesse', 'rage', 'unarmed', 'fear', 'mental', 'poison', …

SelectMultiple(description='Not', options=('agile', 'finesse', 'rage', 'unarmed', 'fear', 'mental', 'poison', …

Checkbox(value=False, description='Add a label?', indent=False)

Checkbox(value=False, description='Add a Name?', indent=False)

Button(description='Create Rule Element', style=ButtonStyle())